In [6]:
!pip install -q scikit-learn pyproj seaborn pandas seisbench pyproj obspy

    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip -q install pyocto

    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
try:
    import obspy
    obspy.read()
except TypeError:
    # Needs to restart the runtime once, because obspy only works properly after restart.
    print('Stopping RUNTIME. If you run this code for the first time, this is expected. Colaboratory will restart automatically. Please run again.')
    exit()

In [11]:
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from pyproj import CRS, Transformer
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch
#pd.set_option('future.no_silent_downcasting', True)
import seisbench.models as sbm

sns.set(font_scale=1.2)
sns.set_style("ticks")

In [12]:
# Projections
wgs84 = CRS.from_epsg(4326)
local_crs = CRS.from_epsg(9155)  # SIRGAS-Chile 2016 / UTM zone 19S
transformer = Transformer.from_crs(wgs84, local_crs)

In [13]:
client = Client("GFZ")

t0 = UTCDateTime("2014/05/01 00:00:00")
t1 = t0 + 12 * 60 * 60
# t1 = t0 + 24 * 60 * 60   # Full day, requires more memory
stream = client.get_waveforms(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

inv = client.get_stations(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

In [25]:
config={}
config["dims"] = ['x(km)', 'y(km)', 'z(km)']
config["use_dbscan"] = True
config["x(km)"] = (250, 600)
config["y(km)"] = (7200, 8000)
config["vel"] = {"P": 7.0, "S": 7.0 / 1.75} 
config["z(km)"] = (0, 200)

config["vel"] = {"p": 7.0, "s": 7.0 / 1.75}  # We assume rather high velocities as we expect deeper events
config["method"] = "BGMM"
if config["method"] == "BGMM":
    config["oversample_factor"] = 4
if config["method"] == "GMM":
    config["oversample_factor"] = 1

# DBSCAN
config["bfgs_bounds"] = (
    (config["x(km)"][0] - 1, config["x(km)"][1] + 1),  # x
    (config["y(km)"][0] - 1, config["y(km)"][1] + 1),  # y
    (0, config["z(km)"][1] + 1),  # x
    (None, None),  # t
)
config["dbscan_eps"] = 25  # seconds
config["dbscan_min_samples"] = 3

# Filtering
config["min_picks_per_eq"] = 10

config['min_p_picks_per_eq']=4
config['min_s_picks_per_eq']=4
config["max_sigma11"] = 2.0
config["max_sigma22"] = 1.0
config["max_sigma12"] = 1.0
config["use_amplitude"]=False
        

In [15]:
import seisbench
import torch
torch.set_num_threads(1)
seisbench.use_backup_repository()
picker = sbm.PhaseNet.from_pretrained("instance")



2024-11-16 01:06:37,914 | seisbench | WARNING | Setting remote root to: https://seisbench.gfz-potsdam.de/mirror/
Please note that this can affect your download speed.


In [16]:
npicks_list=[]
harparesults_list=[]
for r in [1,0.5,0.1,0.05,0.01]:
    picks = picker.classify(stream, batch_size=256, P_threshold=0.075*r, S_threshold=0.1*r).picks
      # Output number of P and S picks
    npicks_list.append(len(picks))
    pick_df = []
    for p in picks:
        pick_df.append({
            "id": p.trace_id,
            "timestamp": p.peak_time.datetime,
            "prob": p.peak_value,
            "type": p.phase.lower()
        })
    pick_df = pd.DataFrame(pick_df)

    station_df = []
    for station in inv[0]:
        station_df.append({
            "id": f"CX.{station.code}.",
            "longitude": station.longitude,
            "latitude": station.latitude,
            "elevation(m)": station.elevation
        })
    station_df = pd.DataFrame(station_df)

    station_df["x(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[0] / 1e3, axis=1)
    station_df["y(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[1] / 1e3, axis=1)
    station_df["z(km)"] = - station_df["elevation(m)"] / 1e3

    station_df["x(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[0] / 1e3, axis=1)
    station_df["y(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[1] / 1e3, axis=1)
    station_df["z(km)"] = - station_df["elevation(m)"] / 1e3

    northing = {station: y for station, y in zip(station_df["id"], station_df["y(km)"])}
    station_dict = {station: (x, y) for station, x, y in zip(station_df["id"], station_df["x(km)"], station_df["y(km)"])}
    
    
    from gamma.utils import association

    catalogs, assignments = association(pick_df, station_df, config, method=config["method"])

    catalog = pd.DataFrame(catalogs)
    assignments = pd.DataFrame(assignments, columns=["pick_idx", "event_idx", "prob_gamma"])


    harparesults_list.append(len(catalog))
    print(r,len(picks),len(catalog),Counter([p.phase for p in picks]))
    

ImportError: Numba needs NumPy 1.24 or less

In [17]:
!pip install --upgrade numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 53.6 MB/s eta 0:00:00:00:0100:01
    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.40.1rc1
    Uninstalling llvmlite-0.40.1rc1:
      Successfully uninstalled llvmlite-0.40.1rc1
  Attempting uninstall: numba
    Found existing installation: numba 0.57.0
    Uninstalling numba-0.57.0:
      Successfully uninstalled numba-0.57.0

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
from gamma.utils import association

catalogs, assignments = association(pick_df, station_df, config, method=config["method"])

catalog = pd.DataFrame(catalogs)
assignments = pd.DataFrame(assignments, columns=["pick_idx", "event_idx", "prob_gamma"])


harparesults_list.append(len(catalog))
print(r,len(picks),len(catalog),Counter([p.phase for p in picks]))

Associating 249 clusters with 32 CPUs
................................................................................................................................................................................
Associated 100 events
.........................................................................1 7399 191 Counter({'P': 4903, 'S': 2496})


In [24]:
config

{'dims': ['x(km)', 'y(km)', 'z(km)'],
 'use_dbscan': True,
 'x(km)': (250, 600),
 'y(km)': (7200, 8000),
 'vel': {'p': 7.0, 's': 4.0},
 'z(km)': (0, 200),
 'method': 'BGMM',
 'oversample_factor': 4,
 'bfgs_bounds': ((249, 601), (7199, 8001), (0, 201), (None, None)),
 'dbscan_eps': 25,
 'dbscan_min_samples': 3,
 'min_picks_per_eq': 5,
 'max_sigma11': 2.0,
 'max_sigma22': 1.0,
 'max_sigma12': 1.0,
 'use_amplitude': False,
 'eikonal': None,
 'ncpu': 32}

 20%|██        | 5/25 [00:32<02:03,  6.19s/it]

In [ ]:
# import pickle

# with open('pick_df.pkl', 'wb') as f:
#     pickle.dump(pick_df, f)
    
# with open('station_df.pkl', 'wb') as f:
#     pickle.dump(station_df, f)
# with open('config.pkl', 'wb') as f:
#     pickle.dump(config, f)
    
    
# import pickle

# # Load pick_df
# with open('pick_df.pkl', 'rb') as f:
#     pick_df = pickle.load(f)

# # Load station_df
# with open('station_df.pkl', 'rb') as f:
#     station_df = pickle.load(f)

# # Load config
# with open('config.pkl', 'rb') as f:
#     config = pickle.load(f)
